<a href="https://colab.research.google.com/github/Tseng0318/yolo_waste/blob/main/garbage_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Verify the path

In [4]:
import os

In [4]:
!ls /content/drive/MyDrive/GARBAGE_CLASSIFICATION

data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [5]:
dataset_path = "/content/drive/MyDrive/GARBAGE_CLASSIFICATION"
splits = ["train", "val", "test"]  # Change "valid" to "val" if needed

def verify():
  '''function to verify the path'''
  for split in splits:
      images_path = os.path.join(dataset_path, split, "images")
      labels_path = os.path.join(dataset_path, split, "labels")

      print(f"\nChecking {split}:")
      print(f"{len(os.listdir(images_path))} images found" if os.path.exists(images_path) else f"No 'images' folder in {split}!")
      print(f"{len(os.listdir(labels_path))} labels found" if os.path.exists(labels_path) else f"No 'labels' folder in {split}!")

In [7]:
# Function to check and clean images & labels
def clean_labels_and_images():
   #'''function to clean and delete any missing labels and images ''''
    for split in splits:
        image_dir = os.path.join(dataset_path, split, "images")
        label_dir = os.path.join(dataset_path, split, "labels")

        if not os.path.exists(image_dir) or not os.path.exists(label_dir):
            print(f"Missing 'images' or 'labels' folder in {split}/")
            continue

        # List all image and label files (without extensions)
        images = {f.rsplit(".", 1)[0] for f in os.listdir(image_dir) if f.endswith((".jpg", ".png"))}
        labels = {f.rsplit(".", 1)[0] for f in os.listdir(label_dir) if f.endswith(".txt")}

        # Find mismatches
        images_without_labels = images - labels
        labels_without_images = labels - images

        print(f"\nChecking {split}/")
        print(f"Found {len(images)} images and {len(labels)} labels")

        # Delete images without labels
        if images_without_labels:
            print(f"Deleting {len(images_without_labels)} images without labels...")
            for img_name in images_without_labels:
                img_path = os.path.join(image_dir, img_name + ".jpg")
                png_path = os.path.join(image_dir, img_name + ".png")
                if os.path.exists(img_path):
                    os.remove(img_path)
                if os.path.exists(png_path):
                    os.remove(png_path)

        # Delete labels without images
        if labels_without_images:
            print(f"Deleting {len(labels_without_images)} labels without images...")
            for lbl_name in labels_without_images:
                lbl_path = os.path.join(label_dir, lbl_name + ".txt")
                if os.path.exists(lbl_path):
                    os.remove(lbl_path)

        print("Cleanup completed!")
# Run the cleanup function
clean_labels_and_images()


Checking train/
Found 7356 images and 7334 labels
Deleting 32 images without labels...
Deleting 10 labels without images...
Cleanup completed!

Checking val/
Found 2098 images and 2098 labels
Cleanup completed!

Checking test/
Found 1042 images and 1042 labels
Cleanup completed!


In [8]:
verify()


Checking train:
7324 images found
7324 labels found

Checking val:
2098 images found
2098 labels found

Checking test:
1042 images found
1042 labels found


In [6]:
import yaml

In [7]:
# Define the expected path
yaml_path = "/content/drive/MyDrive/GARBAGE_CLASSIFICATION/data.yaml"

In [ ]:
# Function to check data.yaml
def check_yaml():
    """Function to check the dataset.yaml file."""
    if not os.path.exists(yaml_path):
        print(f"ERROR: {yaml_path} not found!")
        return

    # Load YAML file
    with open(yaml_path, "r") as f:
        data = yaml.safe_load(f)

    print("\nChecking dataset.yaml...")

    # Check required keys
    required_keys = ["path", "train", "val", "test", "nc", "names"]
    for key in required_keys:
        if key not in data:
            print(f"Missing key in YAML: {key}")

    # Print YAML contents
    print("\n YAML File Contents:")
    for key, value in data.items():
        print(f"{key}: {value}")

    # Check if paths exist
    dataset_root = data.get("path", "")
    if not dataset_root:
        print("Dataset root path is missing!")
        return

    for split in ["train", "val", "test"]:
        split_path = os.path.join(dataset_root, data.get(split, ""))
        if not os.path.exists(split_path):
            print(f"ERROR: {split} path not found -> {split_path}")
        else:
            print(f" {split} path found -> {split_path}")

    print("\nYAML file check completed!")

In [11]:
# Define the correct dataset root path
dataset_root = "/content/drive/MyDrive/GARBAGE_CLASSIFICATION"

# Load the YAML file
yaml_path = f"{dataset_root}/data.yaml"

# Read the existing YAML file
with open(yaml_path, "r") as file:
    data = yaml.safe_load(file)

# Add the missing 'path' key if not present
if "path" not in data:
    data["path"] = dataset_root

# Update paths to absolute paths
data["train"] = f"{dataset_root}/train/images"
data["val"] = f"{dataset_root}/val/images"
data["test"] = f"{dataset_root}/test/images"

# Write the updated YAML file
with open(yaml_path, "w") as file:
    yaml.dump(data, file, default_flow_style=False)

print(f"Updated {yaml_path} successfully!")

Updated /content/drive/MyDrive/GARBAGE_CLASSIFICATION/data.yaml successfully!


In [ ]:
check_yaml()

In [16]:
def check_yaml_print(yaml_path):
    """
    Function to check if the data.yaml file contains all required keys
    and if dataset paths exist. Also, prints dataset information clearly.
    """
    if not os.path.exists(yaml_path):
        print(f"ERROR: {yaml_path} not found!")
        return

    # Load YAML file
    with open(yaml_path, "r") as f:
        data = yaml.safe_load(f)

    print("\nChecking dataset.yaml...\n")

    # Required keys for YOLO dataset
    required_keys = ["path", "train", "val", "nc", "names"]
    missing_keys = [key for key in required_keys if key not in data]

    if missing_keys:
        print(f"Missing keys in YAML: {missing_keys}\n")
    else:
        print("All required keys are present.\n")

    # Display dataset details in the requested format
    print("Dataset Information:")
    print(f"names: {data.get('names', 'Not Found')}")
    print(f"nc: {data.get('nc', 'Not Found')}")
    print(f"path: {data.get('path', 'Not Found')}")
    print(f"train: {data.get('train', 'Not Found')}")
    print(f"val: {data.get('val', 'Not Found')}")
    print(f"test: {data.get('test', 'Not Found (optional)')}")  # test is sometimes missing

    # Check if dataset paths exist
    dataset_root = data.get("path", "")
    if not dataset_root:
        print("\nDataset root path is missing!")
        return

    for split in ["train", "val", "test"]:
        split_path = os.path.join(dataset_root, data.get(split, ""))
        if data.get(split) and not os.path.exists(split_path):
            print(f"ERROR: {split} path not found -> {split_path}")
        elif data.get(split):
            print(f"{split} path found -> {split_path}")

    print("\n✅ YAML file check completed!")



In [17]:
# Run the function with your dataset YAML path
check_yaml_print(yaml_path)  # Update if needed


Checking dataset.yaml...

All required keys are present.

Dataset Information:
names: ['BIODEGRADABLE', 'CARDBOARD', 'GLASS', 'METAL', 'PAPER', 'PLASTIC']
nc: 6
path: /content/drive/MyDrive/GARBAGE_CLASSIFICATION
train: /content/drive/MyDrive/GARBAGE_CLASSIFICATION/train/images
val: /content/drive/MyDrive/GARBAGE_CLASSIFICATION/val/images
test: /content/drive/MyDrive/GARBAGE_CLASSIFICATION/test/images
train path found -> /content/drive/MyDrive/GARBAGE_CLASSIFICATION/train/images
val path found -> /content/drive/MyDrive/GARBAGE_CLASSIFICATION/val/images
test path found -> /content/drive/MyDrive/GARBAGE_CLASSIFICATION/test/images

✅ YAML file check completed!


# Install ultralytics

In [5]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [8]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [21]:
# Download the model
model = YOLO("yolov8m.pt")  # also try 'yolov8s.pt' or 'yolov8n.pt'

100%|██████████| 49.7M/49.7M [00:00<00:00, 244MB/s]


Model summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:16<00:00,  2.00it/s]
                   all       2098      18916       0.58      0.455       0.51      0.351
         BIODEGRADABLE        676      13637      0.796      0.468       0.61       0.34
             CARDBOARD        438       1292      0.712      0.471      0.583       0.44
                 GLASS        805       2380      0.839      0.659      0.764      0.571
                 METAL        398       1360       0.74      0.547      0.639      0.432
                 PAPER         15         33     0.0282     0.0909     0.0694     0.0665
               PLASTIC         91        214      0.364      0.495      0.396      0.257
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_v2
ultralytics.utils.metrics.DetMetrics object with attributes:

In [14]:
# Load the trained model
model = YOLO("/content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_v2/weights/best.pt")

# Freeze the backbone for initial training
model.model.model[0].requires_grad_(False)  # Freezes the first layer (backbone)

# First training phase (Freeze backbone)
model.train(
    data="/content/drive/MyDrive/GARBAGE_CLASSIFICATION/data.yaml",
    epochs=10,  # Train only the head initially
    workers=2,
    imgsz=640,
    batch=32,
    device="cuda",
    optimizer="AdamW",
    lr0=0.002,
    lrf=0.2,
    weight_decay=0.0001,
    warmup_epochs=3.0,  # Shorter warmup since we're using a trained model
    patience=20,
    save=True,
    save_period=5,

    # Reduce Augmentations
    mosaic=0.3,
    hsv_h=0.01,
    hsv_s=0.2,
    hsv_v=0.2,
    translate=0.1,
    scale=0.3,
    fliplr=0.5,

    name="garbage_detection_frozen",
    project="/content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs"
)

# Unfreeze the backbone
model.model.model[0].requires_grad_(True)  # Unfreezes the backbone

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_v2/weights/best.pt, data=/content/drive/MyDrive/GARBAGE_CLASSIFICATION/data.yaml, epochs=10, time=None, patience=20, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=cuda, workers=2, project=/content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs, name=garbage_detection_frozen2, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnosti

train: Scanning /content/drive/MyDrive/GARBAGE_CLASSIFICATION/train/labels.cache... 7324 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7324/7324 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/GARBAGE_CLASSIFICATION/val/labels.cache... 2098 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2098/2098 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_frozen2/labels.jpg... 
optimizer: AdamW(lr=0.002, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0001), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_frozen2
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       2098      18916      0.553      0.469      0.499      0.339



10 epochs completed in 0.239 hours.
Optimizer stripped from /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_frozen2/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_frozen2/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_frozen2/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:16<00:00,  1.99it/s]


                   all       2098      18916      0.551      0.471      0.499      0.339
         BIODEGRADABLE        676      13637      0.736      0.511      0.601       0.33
             CARDBOARD        438       1292      0.648      0.509       0.57      0.433
                 GLASS        805       2380      0.782      0.688      0.768      0.562
                 METAL        398       1360      0.714      0.559      0.625      0.423
                 PAPER         15         33     0.0289     0.0909     0.0695     0.0674
               PLASTIC         91        214      0.396      0.467      0.361       0.22
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_frozen2


Conv(
  (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
)

In [18]:
# Fine-tuning phase (Final tuning of entire model)
model.train(
    data="/content/drive/MyDrive/GARBAGE_CLASSIFICATION/data.yaml",
    epochs=20,  # Run 20 epochs first, monitor improvement
    imgsz=640,
    batch=32,
    workers=2,
    device="cuda",
    optimizer="AdamW",
    lr0=5e-5,  # Reduced LR for precise fine-tuning
    lrf=0.02,  # Slightly lower final LR
    weight_decay=5e-5,  # Lower weight decay to avoid overfitting
    warmup_epochs=1.0,  # Short warmup to stabilize training
    patience=10,  # Stop early if no improvement
    save=True,
    save_period=5,  # Save every 5 epochs
    verbose=True,  # Log detailed output

    # Further Reduce Augmentations (Stable Training)
    mosaic=0.05,  # Minimal mosaic to reduce variance
    translate=0.02,  # Lower translation impact
    scale=0.1,  # Less aggressive scaling
    fliplr=0.2,  # Reduce excessive flipping

    name="garbage_detection_fine_tune_unfrozen",
    project="/content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs"
)

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_v2/weights/best.pt, data=/content/drive/MyDrive/GARBAGE_CLASSIFICATION/data.yaml, epochs=20, time=None, patience=10, batch=32, imgsz=640, save=True, save_period=5, cache=False, device=cuda, workers=2, project=/content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs, name=garbage_detection_fine_tune_unfrozen3, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=Fa

train: Scanning /content/drive/MyDrive/GARBAGE_CLASSIFICATION/train/labels.cache... 7324 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7324/7324 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/GARBAGE_CLASSIFICATION/val/labels.cache... 2098 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2098/2098 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_fine_tune_unfrozen3/labels.jpg... 
optimizer: AdamW(lr=5e-05, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=5e-05), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_fine_tune_unfrozen3
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       2098      18916      0.547      0.446      0.467      0.316
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 3, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



13 epochs completed in 0.298 hours.
Optimizer stripped from /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_fine_tune_unfrozen3/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_fine_tune_unfrozen3/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_fine_tune_unfrozen3/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:16<00:00,  2.00it/s]


                   all       2098      18916      0.579      0.432      0.475       0.32
         BIODEGRADABLE        676      13637      0.751      0.466      0.566      0.303
             CARDBOARD        438       1292      0.695      0.473      0.552      0.417
                 GLASS        805       2380      0.865      0.643       0.75      0.554
                 METAL        398       1360      0.756      0.501        0.6      0.399
                 PAPER         15         33     0.0296     0.0909     0.0718     0.0681
               PLASTIC         91        214      0.378      0.421      0.308       0.18
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /content/drive/MyDrive/GARBAGE_CLASSIFICATION/runs/garbage_detection_fine_tune_unfrozen3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x795af45446d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     